In [ ]:

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql import functions as f\n",
    "\n",
    "import pandas as pd\n",
    "from IPython.core.display import display\n",
    "import seaborn as sns\n",
    "\n",
    "\n",
    "spark = SparkSession.builder.getOrCreate()\n",
    "\n",
    "# General settings for display purposes\n",
    "pd.options.display.max_columns = None\n",
    "pd.options.display.max_rows = None\n",
    "pd.options.display.max_colwidth = 144\n",
    "sns.set(color_codes=True)\n",
    "\n",
    "# Source sentiment140: http://help.sentiment140.com/for-students/\n",
    "schema = \"polarity FLOAT, id LONG, date_time STRING, query STRING, user STRING, text STRING\"\n",
    "spark_reader = spark.read.schema(schema)\n",
    "\n",
    "# file 1: testdata.manual.2009.06.14.csv\n",
    "TESTDATA_PATH = (\n",
    "    \"/home/jovyan/data-sets/sentiment-140-training-data/testdata.manual.2009.06.14.csv\"\n",
    ")\n",
    "raw_test_data = spark_reader.csv(\n",
    "    TESTDATA_PATH,\n",
    "    quote='\"',\n",
    "    header=False,\n",
    "    inferSchema=True,\n",
    "    columnNameOfCorruptRecord=\"corrupt_data\",\n",
    ").cache()\n",
    "\n",
    "# file 2: training.1600000.processed.noemoticon.csv\n",
    "TRAININGDATA_PATH = \"/home/jovyan/data-sets/sentiment-140-training-data/training.1600000.processed.noemoticon.csv\"\n",
    "raw_training_data = spark_reader.csv(\n",
    "    TRAININGDATA_PATH,\n",
    "    quote='\"',\n",
    "    header=False,\n",
    "    inferSchema=True,\n",
    "    columnNameOfCorruptRecord=\"corrupt_data\",\n",
    ").cache()\n",
    "\n",
    "# path that we will write our raw data to\n",
    "OUTPUT_PATH = \"/home/jovyan/data-sets/sentiment-140-training-data/RAW\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# First look at the test data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Count of data\n",
    "print(f\"Overall data count: {raw_test_data.count()}\")\n",
    "\n",
    "# Data summary\n",
    "display(raw_test_data.summary().toPandas())\n",
    "print(\"Data schema\")\n",
    "raw_test_data.printSchema()\n",
    "\n",
    "# Let's look at 50 rows of data\n",
    "display(raw_test_data.limit(50).toPandas())\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# First look at the training data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Count of data\n",
    "print(f\"Overall data count: {raw_training_data.count()}\")\n",
    "\n",
    "# Data summary\n",
    "display(raw_training_data.summary().toPandas())\n",
    "print(\"Data schema\")\n",
    "raw_training_data.printSchema()\n",
    "\n",
    "# Let's look at 50 rows of data\n",
    "display(raw_training_data.limit(50).toPandas())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Test data:\n",
    "- 498 rows of test_data\n",
    "\n",
    "Training data:\n",
    "- 1600000 rows of training_data\n",
    "\n",
    "\n",
    "### Initial Findings:\n",
    "- We need to apply a proper schema\n",
    "- The date column needs fixing\n",
    "- We need to extract twitter user names/handles (we'll extract it and call the output column `users_mentioned`)\n",
    "- We need to extract hashtags and replace them with the words from the hashtag (we'll extract it and call the output column `hashtags`)\n",
    "- We need to extract URLs, as our algorithm won't need that or use that (we'll simply remove it from the data)\n",
    "- The same goes for email-address\n",
    "- HTML does not appear properly unescaped, we're going to have to fix that (example: `&lt;3` and `s&amp;^t`)\n",
    "- Encoding seems to be 'broken' (example: `�����ߧ�ǿ�����ж�؜��� &lt;&lt;----I DID NOT KNOW I CUD or HOW TO DO ALL DAT ON MY PHONE TIL NOW. WOW..MY LIFE IS NOW COMPLETE. JK.`)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Detailed statistics\n",
    "\n",
    "## Polarity\n",
    "According to Sentiment140 documentation, we would expect the `polarity` column to have one of three values representing user sentiment:\n",
    "- 0 = negative\n",
    "- 2 = neutral\n",
    "- 4 = positive\n",
    "\n",
    "Once we train our own model, we don't want data-skew to introduce bias. So let's see how polarity is distributed in the data that we have.\n",
    "\n",
    "#### Polarity column (test data)\n",
    "Let's first look at the test data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = raw_test_data.select(\"polarity\").na.drop()\n",
    "print(f\"No of rows with Polarity: {df.count()}/{raw_test_data.count()}\")\n",
    "\n",
    "sns.distplot(df.toPandas())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Polarity column (training data)\n",
    "\n",
    "Now let's look at the training data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = raw_training_data.select(\"polarity\").na.drop()\n",
    "print(f\"No of rows with Polarity: {df.count()} / {raw_training_data.count()}\")\n",
    "\n",
    "sns.distplot(df.toPandas())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Results:\n",
    "We can clearly see that the training data only has polarity data centered around 0 (Negative) and 4 (Positive).\n",
    "\n",
    "Let's confirm this:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "polarity_df = raw_training_data.select(\"polarity\").cache()\n",
    "\n",
    "polarity_df.groupBy(\"polarity\").count().toPandas()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Very nice! We have a nice even 50/50 split between polarity."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Conclusions:\n",
    "- As 498 rows is way too little for us to train a model on, we're going to disregard this dataset and focus on the Training Data. \n",
    "- We've determined the steps that need to be taken to clean the data"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Store our raw data\n",
    "\n",
    "Now it's time for us to write the raw data we intend to use to disk.  \n",
    "We're going to:\n",
    "- keep the format CSV\n",
    "- partition the data by polarity, this will create 2 subfolders inside our output folder \n",
    "- repartition the data in 20 partitions: This will ensure that we have 20 smaller csv files per partition"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "raw_training_data.repartition(20).write.partitionBy(\"polarity\").csv(\n",
    "    OUTPUT_PATH, mode=\"overwrite\"\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
